# BlueSky Device for APS taxi & fly scans

Some EPICS fly scans at APS are triggered by a pair of EPICS *busy* records.  The *busy* record is set, which triggers the external controller to do the fly scan and then reset the *busy* record.  The first *busy* is called `taxi` and is responsible for preparing the hardware to fly.  The second *busy* performs the actual fly scan.  In a third (optional) phase, data is collected from hardware and written to a file.


## Python imports and definitions <a class="anchor" id="imports" />

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd and BluSky packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the BlueSky RunEngine.
* Create an instance of the databroker using the `mongodb_config.yml` file on the local machine
* Arrange for the databroker to receive all events from the RunEngine

In [1]:
import enum
import epics
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

FORMAT = '%(asctime)s %(levelname)s %(name)s - %(message)s'
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO, format=FORMAT)

RE = bluesky.RunEngine({})
db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)
RE.msg_hook = bluesky.utils.ts_msg_hook

## EPICS setup

A custom EPICS database has been created to develop this support.  The database is loaded on IOC startup with this command:

    dbLoadRecords("taxi_fly.db","P=prj:tf:")

file: `taxi_fly.db`

```
record(busy, "$(P)taxi")
{
	field(DESC, "prepare for fly scan")
	field(OUT, "$(P)do_taxi.PROC")
}
record(sseq, "$(P)do_taxi")
{
	field(DESC, "sequence of taxi steps")
    # the only thing we do here is quit after a short time
	field(DLY1, 5.0)
	field(STR1, "Done")
	field(LNK1, "$(P)taxi CA NMS")
}

record(busy, "$(P)fly")
{
	field(DESC, "execute fly scan")
	field(OUT, "$(P)do_fly.PROC")
}
record(sseq, "$(P)do_fly")
{
	field(DESC, "sequence of fly steps")
    # the only thing we do here is quit after a short time
	field(DLY1, 5.0)
	field(STR1, "Done")
	field(LNK1, "$(P)fly CA NMS")
}
```

## Define the TaxiFlyScanDevice

In [2]:
class TaxiFlyScanDevice(ophyd.Device):
    taxi = ophyd.Component(ophyd.EpicsSignal, "taxi", put_complete=True)
    fly = ophyd.Component(ophyd.EpicsSignal, "fly", put_complete=True)
    
    def plan(self):
        logger.info("before taxi")
        yield from bluesky.plan_stubs.mv(self.taxi, self.taxi.enum_strs[1])
        logger.info("after taxi")
        
        logger.info("before fly")
        yield from bluesky.plan_stubs.mv(self.fly, self.fly.enum_strs[1])
        logger.info("after fly")

In [3]:
ifly = TaxiFlyScanDevice("prj:tf:", name="ifly")

In [4]:
ifly.taxi.put(0)

In [5]:
RE(ifly.plan())

2018-05-14 14:10:10,156 INFO root - before taxi


14:10:10.161009 set               -> ifly_taxi       args: ('Busy',), kwargs: {'group': 'f4906b42-af21-439a-8df8-c5e95d7ca268'}
14:10:10.164069 wait              -> None            args: (), kwargs: {'group': 'f4906b42-af21-439a-8df8-c5e95d7ca268'}


2018-05-14 14:10:16,163 INFO root - after taxi
2018-05-14 14:10:16,170 INFO root - before fly


14:10:16.184975 set               -> ifly_fly        args: ('Busy',), kwargs: {'group': '9715fa03-6852-4ffa-92ae-5471eea91cdd'}
14:10:16.186176 wait              -> None            args: (), kwargs: {'group': '9715fa03-6852-4ffa-92ae-5471eea91cdd'}


2018-05-14 14:10:26,153 INFO root - after fly


()